### Student Information
Name: 蕭靖澂 Ching-Cheng Hsiao

Student ID: 110033632

GitHub ID: hsiaooo

Kaggle name: chingchenghsiao

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2021-Lab2-master Repo](https://github.com/fhcalderon87/DM2021-Lab2-master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/c/dm2021-lab2-hw2/) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 24th 11:59 pm, Friday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 29th 11:59 pm, Wednesday)__. 

In [1]:
### Begin Assignment Here

### 1. Load kaggle dataset

In [4]:
# load the libraries
import os
import json
import pandas as pd
import numpy as np

# load the kaggle dataset
path       = os.path.dirname(os.path.abspath("tweets_DM.json"))
raw_data = pd.read_json(path+"/dataset/tweets_DM.json",lines=True)
tweets   = pd.json_normalize(data=raw_data['_source'])
identify = pd.read_csv(path+"/dataset/data_identification.csv")
emotion  = pd.read_csv(path+"/dataset/emotion.csv")

In [10]:
tweets.head()

,tweet.hashtags,tweet.tweet_id,tweet.text
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,[],0x2de201,"""Trust is not the same as faith. A friend is s..."


In [11]:
# deal with emotion, seperate the id and emotion
emotion_list = []
id_list = []
for i, j in zip(emotion['emotion'], emotion['tweet_id']):
    if ',' in i:
        i = i.split(',')[1]
    emotion_list.append(i)
    id_list.append(j)
    
print(emotion_list[:50])
print(id_list[:50])

emotion_dict = {'tweet_id':id_list, 'emotion':emotion_list}
emotion_2 = pd.DataFrame(emotion_dict)

['sadness', 'disgust', 'anticipation', 'joy', 'anticipation', 'joy', 'sadness', 'sadness', 'trust', 'sadness', 'anticipation', 'joy', 'sadness', 'joy', 'joy', 'joy', 'joy', 'joy', 'sadness', 'disgust', 'trust', 'anticipation', 'sadness', 'anger', 'joy', 'trust', 'joy', 'fear', 'joy', 'sadness', 'joy', 'anticipation', 'trust', 'joy', 'joy', 'joy', 'disgust', 'joy', 'joy', 'joy', 'anticipation', 'sadness', 'disgust', 'joy', 'joy', 'trust', 'sadness', 'joy', 'anger', 'fear']
['0x3140b1', '0x368b73', '0x296183', '0x2bd6e1', '0x2ee1dd', '0x34cd80', '0x33f099', '0x2ae7b7', '0x2408d4', '0x2b193b', '0x215b2e', '0x349b83', '0x270297', '0x213b5e', '0x31a0e7', '0x28a0d1', '0x2c8770', '0x317e52', '0x30188c', '0x361411', '0x1e16a0', '0x255135', '0x203527', '0x27b6b2', '0x213263', '0x255bbd', '0x33a6b7', '0x30e5c4', '0x2bffa3', '0x1dfa03', '0x369a1e', '0x37db4e', '0x1c9da9', '0x1c91b2', '0x1d5ec5', '0x38b81e', '0x25e867', '0x383ed2', '0x1e539d', '0x346f76', '0x1d7368', '0x2b732b', '0x31bb2a', '0x1f8

In [12]:
emotion_2.head()

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation


In [13]:
# rename column names
tweets = tweets.rename(index=str, columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id", "tweet.hashtags":"hashtags"})

# add identify tags to dataframe
tweets = pd.merge(tweets, identify, on="tweet_id")

print(tweets.shape)
tweets.head()

(1867535, 4)


,hashtags,tweet_id,text,identification
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test


In [14]:
# remove emoji and punctuation
from string import punctuation
import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def remove_punctuation_emoji(value):
    value = remove_emojis(value)
    return ''.join([c for c in value if (c not in punctuation)])

In [15]:
# remove the emoji, punctuation and LH, which in a lot of text, and make all word be lower
from tqdm import tqdm

for i, text in enumerate(tqdm(tweets['text'])):
    tweets['text'][i] = remove_punctuation_emoji(text).replace("LH", "").replace("  ", " ").lower()    

100%|███████████████████████████████████████| 1867535/1867535 [11:45:27<00:00, 44.12it/s]


In [16]:
print(tweets['text'][3])

now issa is stalking tasha 


In [17]:
# get training set and test set
train_df = tweets[tweets["identification"] == "train"]
test_df  = tweets[tweets["identification"] == "test"]

# drop identification tags
train_df.drop(columns=["identification"], inplace=True)
test_df.drop(columns=["identification"], inplace=True)

# drop hashtags
train_df.drop(columns=["hashtags"], inplace=True)
test_df.drop(columns=["hashtags"], inplace=True)

print(f'Training set\n{train_df.head()}\n')
print(f'Testing set\n{test_df.head()}')

Training set
   tweet_id                                               text
0  0x376b20  people who post add me on snapchat must be deh...
1  0x2d5350  brianklaas as we see trump is dangerous to fre...
3  0x1cd5b0                        now issa is stalking tasha 
5  0x1d755c  riskshow thekevinallison thx for the best time...
6  0x2c91a8            still waiting on those supplies liscus 

Testing set
    tweet_id                                               text
2   0x28b412  confident of your obedience i write to you kno...
4   0x2de201  trust is not the same as faith a friend is som...
9   0x218443  when do you have enough when are you satisfied...
30  0x2939d5  god woke you up now chase the day godsplan god...
33  0x26289a  in these tough times who do you turn to as you...


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
# add emotion column
train_df = pd.merge(train_df, emotion_2, on="tweet_id")
test_df = test_df.assign(emotion="")

In [ ]:
# use tweet_id as index
train_df.set_index("tweet_id",inplace=True)
test_df.set_index("tweet_id",inplace=True)

train_df.head()

In [7]:
test_df.head()

,text,emotion
tweet_id,,
0x28b412,confident of your obedience i write to you kno...,
0x2de201,trust is not the same as faith a friend is som...,
0x218443,when do you have enough when are you satisfied...,
0x2939d5,god woke you up now chase the day godsplan god...,
0x26289a,in these tough times who do you turn to as you...,


In [22]:
# save to pickle file
train_df.to_pickle("pkl/train_df_clean.pkl")
test_df.to_pickle("pkl/test_df_clean.pkl")

In [27]:
# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df_clean.pkl")
test_df  = pd.read_pickle(path+"/pkl/test_df_clean.pkl")

In [6]:
print("Train_df\n", train_df)
print("\nTest_df\n", test_df)

Train_df
                                                        text       emotion
tweet_id                                                                 
0x376b20  people who post add me on snapchat must be deh...  anticipation
0x2d5350  brianklaas as we see trump is dangerous to fre...       sadness
0x1cd5b0                        now issa is stalking tasha           fear
0x1d755c  riskshow thekevinallison thx for the best time...           joy
0x2c91a8            still waiting on those supplies liscus   anticipation
...                                                     ...           ...
0x321566  im so happy nowonder the name of this show hap...           joy
0x38959e  in every circumtance id like to be thankful to...           joy
0x2cbca6  theres currently two girls walking around the ...           joy
0x24faed  ah corporate life where you can date using jus...           joy
0x34be8c                  blessed to be living sundayvibes            joy

[1455563 rows x 2 columns]


### 2. Pre-processing

#### 2.1 Tf-idf

*In Tf-idf, I used max_features=20000, 30000, 50000.*

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
# from nltk.tokenize import word_tokenize
# from nltk.tokenize.stanford import StanfordTokenizer

tknzr = TweetTokenizer(preserve_case=False)
# tknzr = word_tokenize()
# tknzr = StanfordTokenizer()
tfidf = TfidfVectorizer(max_features=50000, stop_words='english', tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_features=50000, stop_words='english',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f68cea63280>>)

In [29]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

(1455563, 50000)

In [30]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

(411972, 50000)

In [31]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [32]:
print(y_test[0:4])

tweet_id
0x28b412    
0x2de201    
0x218443    
0x2939d5    
Name: emotion, dtype: object


#### 2.2 Bag of words

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW_vectorizer = CountVectorizer() 

In [31]:
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(train_df['text'])

# 2. Transform documents to document-term matrix.
train_data_BOW_features = BOW_vectorizer.transform(train_df['text'])
test_data_BOW_features = BOW_vectorizer.transform(test_df['text'])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1208: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [32]:
# check the result
train_data_BOW_features

<1455563x904857 sparse matrix of type '<class 'numpy.int64'>'
	with 17414299 stored elements in Compressed Sparse Row format>

In [33]:
type(train_data_BOW_features)

scipy.sparse.csr.csr_matrix

In [34]:
# check the dimension
train_data_BOW_features.shape

(1455563, 904857)

In [35]:
# observe some feature names
feature_names = BOW_vectorizer.get_feature_names_out()
feature_names[:10]

array(['00', '000', '0000', '00000', '000000', '0000000', '000000000000',
       '00000000000000', '000000000000000001', '0000000000001'],
      dtype=object)

In [36]:
import nltk

# build analyzers (bag-of-words)
BOW_20000 = CountVectorizer(max_features=20000, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_20000.fit(train_df['text'])

train_data_BOW_features_20000 = BOW_20000.transform(train_df['text'])

## check dimension
train_data_BOW_features_20000.shape

(1455563, 20000)

In [37]:
# observe some feature names
feature_names_20000 = BOW_20000.get_feature_names()
feature_names_20000[100:110]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000000',
 '1000am',
 '1000pm',
 '1000s',
 '100daysofgratitude']

### 3. Train model

#### 3.1 Logistic regression

In [38]:
# copy the X_train, X_test, y_train, y_test for Logistic regression
lr_X_train = X_train
lr_X_test  = X_test
lr_y_train = y_train
lr_y_test  = y_test

In [42]:
from sklearn.linear_model import LogisticRegression

# fit the X_train and y_train
lr = LogisticRegression(C=6, n_jobs=-1, max_iter=1000)
lr.fit(lr_X_train,lr_y_train)

LogisticRegression(C=6, max_iter=1000, n_jobs=-1)

In [43]:
# predict the result
pred_result_lr = lr.predict(lr_X_test)
pred_result_lr.shape

(411972,)

In [44]:
import pickle

# save the model
f = open('model/lr_tfidf50000_clean.pickle', 'wb')
pickle.dump(lr, f)
f.close()

In [45]:
# save the result
test_df['emotion']=pred_result_lr
test_df.drop(columns=['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/lr_tfidf50000_clean.csv')
print('Saved file to ', path, '/submission/lr_tfidf50000_clean.csv')

Saved file to  /home/neaf-2070/code/hsiao/data_mining/DMLab2/DM2021-Lab2-HW2 /submission/lr_tfidf50000_clean.csv


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


I used the GridSearchCV to find the best parameters of Logistic Regression, and the result is C = 10000, penalty = "l2", but if I use C=1000, penalty="l2", submit to kaggle, the score only 0.45819, less than use C=6, score is 0.45963.

#### 3.2 XGBoost Classifier

In [40]:
# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df_clean.pkl")
test_df  = pd.read_pickle(path+"/pkl/test_df_clean.pkl")

# copy the X_train, X_test, y_train, y_test for XGBoost
xgb_X_train = X_train
xgb_X_test  = X_test
xgb_y_train = y_train
xgb_y_test  = y_test

# take a look at data dimension 
print('X_train.shape: ', xgb_X_train.shape)
print('y_train.shape: ', xgb_y_train.shape)
print('X_test.shape: ', xgb_X_test.shape)
print('y_test.shape: ', xgb_y_test.shape)

X_train.shape:  (1455563, 50000)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 50000)
y_test.shape:  (411972,)


In [46]:
# label encoding the y_train to transfer 8 emotions to 0-7
emotion = pd.Categorical(xgb_y_train.iloc[:], categories=xgb_y_train.iloc[:].unique(), ordered=True)

xgb_y_train.iloc[:] = emotion.codes
print(xgb_y_train)

# see each emotion label
print('anger \t\t= ', xgb_y_train[9])
print('anticipation \t= ', xgb_y_train[0])
print('disgust \t= ', xgb_y_train[18])
print('fear \t\t= ', xgb_y_train[2])
print('joy \t\t= ', xgb_y_train[3])
print('sadness \t= ', xgb_y_train[1])
print('surprise \t= ', xgb_y_train[19])
print('trust \t\t= ', xgb_y_train[15])

tweet_id
0x376b20    0
0x2d5350    1
0x1cd5b0    2
0x1d755c    3
0x2c91a8    0
           ..
0x321566    3
0x38959e    3
0x2cbca6    3
0x24faed    3
0x34be8c    3
Name: emotion, Length: 1455563, dtype: object
anger 		=  4
anticipation 	=  0
disgust 	=  6
fear 		=  2
joy 		=  3
sadness 	=  1
surprise 	=  7
trust 		=  5


In [47]:
import xgboost as xgb

# fit the X_train and y_train
xgbc = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgbc.fit(xgb_X_train, xgb_y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...)

In [48]:
# predict the result
pred_result_xgbc = xgbc.predict(xgb_X_test)

pred_result_xgbc.shape
print(pred_result_xgbc)

[3 0 3 ... 3 3 3]


In [49]:
print(pred_result_xgbc[-10:])

[3 3 3 1 3 3 0 3 3 3]


In [50]:
# transfer the lable to emotion
pred_result = []
for i, emo in enumerate(pred_result_xgbc):
    if emo == 0:
        pred_result.append('anticipation')
    elif emo == 1:
        pred_result.append('sadness')
    elif emo == 2:
        pred_result.append('fear')
    elif emo == 3:
        pred_result.append('joy')
    elif emo == 4:
        pred_result.append('anger')
    elif emo == 5:
        pred_result.append('trust')
    elif emo == 6:
        pred_result.append('disgust')
    elif emo == 7:
        pred_result.append('surprise')

In [51]:
pred = np.array(pred_result)
print(pred[-10:])

['joy' 'joy' 'joy' 'sadness' 'joy' 'joy' 'anticipation' 'joy' 'joy' 'joy']


In [52]:
# save the model
f = open('model/xgbc_tfid_clean.pickle', 'wb')
pickle.dump(xgbc, f)
f.close()

In [55]:
# save the result
test_df['emotion'] = pred
test_df.drop(columns = ['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns = ['emotion']
test_df.to_csv(path+'/submission/xgbc_clean.csv')
print('Saved file to ', path, '/submission/xgbc_clean.csv')

Saved file to  /home/neaf-2070/code/hsiao/data_mining/DMLab2/DM2021-Lab2-HW2 /submission/xgbc_clean.csv


#### 3.3 Decision Tree

In [33]:
from sklearn.tree import DecisionTreeClassifier

# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df_clean.pkl")
test_df  = pd.read_pickle(path+"/pkl/test_df_clean.pkl")

# for a classificaiton problem, you need to provide both training & testing data
DT_X_train = X_train
DT_y_train = y_train
DT_X_test = X_test
DT_y_test = y_test

# take a look at data dimension 
print('X_train.shape: ', DT_X_train.shape)
print('y_train.shape: ', DT_y_train.shape)
print('X_test.shape: ', DT_X_test.shape)
print('y_test.shape: ', DT_y_test.shape)

X_train.shape:  (1455563, 50000)
y_train.shape:  (1455563,)
X_test.shape:  (411972, 50000)
y_test.shape:  (411972,)


In [34]:
# label encoding the y_train to transfer 8 emotions to 0-7
emotion = pd.Categorical(DT_y_train.iloc[:], categories=DT_y_train.iloc[:].unique(), ordered=True)

DT_y_train.iloc[:] = emotion.codes
DT_y_train = DT_y_train.astype('int')
print(DT_y_train)

tweet_id
0x376b20    0
0x2d5350    1
0x1cd5b0    2
0x1d755c    3
0x2c91a8    0
           ..
0x321566    3
0x38959e    3
0x2cbca6    3
0x24faed    3
0x34be8c    3
Name: emotion, Length: 1455563, dtype: int64


In [ ]:
# build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0, verbose=2)

# training!
DT_model = DT_model.fit(DT_X_train, DT_y_train)

# predict!
y_test_pred = DT_model.predict(DT_X_test)

# so we get the pred result
y_test_pred[:10]

In [ ]:
# transfer the lable to emotion
pred_result = []
for i, emo in enumerate(y_test_pred):
    if emo == 0:
        pred_result.append('anticipation')
    elif emo == 1:
        pred_result.append('sadness')
    elif emo == 2:
        pred_result.append('fear')
    elif emo == 3:
        pred_result.append('joy')
    elif emo == 4:
        pred_result.append('anger')
    elif emo == 5:
        pred_result.append('trust')
    elif emo == 6:
        pred_result.append('disgust')
    elif emo == 7:
        pred_result.append('surprise')

In [ ]:
# save the model
f = open('model/DT_tfidf_clean.pickle', 'wb')
pickle.dump(DT_model, f)
f.close()

In [ ]:
# save the result
test_df['emotion']= y_test_pred
test_df.drop(columns=['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/DT_tfidf_cleancsv')
print('Saved file to ', path, '/submission/DT_tfidf_clean.csv')

#### 3.4 LSTM

In [13]:
from keras.models import Model
from keras.utils import np_utils
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import rmsprop_v2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

2021-12-26 14:04:33.118733: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-26 14:04:33.118806: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
# load a pickle file
train_df = pd.read_pickle(path+"/pkl/train_df_clean.pkl")
test_df = pd.read_pickle(path+"/pkl/test_df_clean.pkl")

# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [15]:
# set the max_words to tokenizer
max_words = 20000
max_len = 300
tok = Tokenizer(num_words=max_words)

# fitting
tok.fit_on_texts(train_df['text'])

In [16]:
# give the sequences
train_seq = tok.texts_to_sequences(train_df['text'])
test_seq = tok.texts_to_sequences(test_df['text'])

train_seq_mat = sequence.pad_sequences(train_seq, maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq, maxlen=max_len)

print(train_seq_mat.shape)
print(test_seq_mat.shape)

(1455563, 300)
(411972, 300)


In [17]:
from sklearn.preprocessing import LabelEncoder

# label encoding
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    '''
    label encoding
    
    :param le: LabelEncoder() function
    :param labels: which one want to encoder
    '''
    enc = le.transform(labels)
    return np_utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    '''
    label decoding
    
    :param le: LabelEncoder() function
    :param labels: which one want to decoer
    '''
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

# encoding the y_train and y_test
y_train = label_encode(label_encoder, y_train)
# y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 tweet_id
0x376b20    anticipation
0x2d5350         sadness
0x1cd5b0            fear
0x1d755c             joy
Name: emotion, dtype: object

y_train.shape:  (1455563,)
y_test.shape:  (411972,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1455563, 8)
y_test.shape:  (411972,)


In [18]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  50000
output_shape:  8


In [19]:
inputs = Input(name='inputs',shape=[max_len])
## Embedding
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = LSTM(128)(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(output_shape,activation="softmax",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss="categorical_crossentropy",optimizer=rmsprop_v2.RMSprop(),metrics=["accuracy"])

2021-12-26 14:05:11.337850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-26 14:05:11.339422: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-26 14:05:11.339674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-26 14:05:11.339890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-26 14:05:11.376410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 128)          2560128   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 FC1 (Dense)                 (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 FC2 (Dense)                 (None, 8)                 1032      
                                                                 
Total params: 2,709,256
Trainable params: 2,709,256
Non-train

In [20]:
model_fit = model.fit(train_seq_mat, y_train,batch_size=128, epochs=3, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)] )

2021-12-26 14:05:11.597743: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1746675600 exceeds 10% of free system memory.


Epoch 1/3
11372/11372 [==============================] - 2199s 193ms/step - loss: 1.3236 - accuracy: 0.5281
Epoch 2/3
11372/11372 [==============================] - 2199s 193ms/step - loss: 1.2398 - accuracy: 0.5615
Epoch 3/3
11372/11372 [==============================] - 2205s 194ms/step - loss: 1.2200 - accuracy: 0.5705


#### *Train epoch 5 times:*
Epoch 1/5</br>
11372/11372 [==============================] - 3087s 271ms/step - loss: 1.2539 - accuracy: 0.5520</br>
Epoch 2/5</br>
11372/11372 [==============================] - 3091s 272ms/step - loss: 1.1417 - accuracy: 0.5952</br>
Epoch 3/5</br>
11372/11372 [==============================] - 3088s 272ms/step - loss: 1.1145 - accuracy: 0.6066</br>
Epoch 4/5</br>
11372/11372 [==============================] - 3088s 272ms/step - loss: 1.1034 - accuracy: 0.6127</br>
Epoch 5/5</br>
11372/11372 [==============================] - 3090s 272ms/step - loss: 1.0969 - accuracy: 0.6166</br>

*Acutally in epoch 5, the accuracy is decreasing gradually, it's mean that this result is overfitting, so I finally choose epoch 3 to training.*

In [21]:
# predict the result using our model
pred_result_lstm = label_decode(label_encoder, model.predict(test_seq_mat, batch_size=128))
pred_result_lstm[:5]

2021-12-26 15:55:15.723784: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 494366400 exceeds 10% of free system memory.


array(['anticipation', 'anticipation', 'joy', 'anticipation', 'trust'],
      dtype=object)

In [22]:
# save the model
from keras.models import load_model
model.save('model_clean.h5')  # creates a HDF5 file 'model.h5'

In [24]:
# save the result
test_df['emotion']=pred_result_lstm
test_df.drop(columns=['text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv(path+'/submission/keras_tfidf_clean.csv')
print('Saved file to ', path, '/submission/keras_tfidf_clean.csv')

Saved file to  /home/neaf-2070/code/hsiao/data_mining/DMLab2/DM2021-Lab2-HW2 /submission/keras_tfidf_clean.csv
